In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Wed Jun 30 13:21:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!unzip -qq '/content/drive/MyDrive/food-101.zip'

In [4]:
%%capture
import os
from collections import defaultdict
import shutil, sys 

if not os.path.isdir('/content/food-101/test') or not os.path.isdir('/content/food-101/train'):

    def copytree(src, dst, symlinks = False, ignore = None):
        if not os.path.exists(dst):
            os.makedirs(dst)
            shutil.copystat(src, dst)
        lst = os.listdir(src)
        if ignore:
            excl = ignore(src, lst)
            lst = [x for x in lst if x not in excl]
        for item in lst:
            s = os.path.join(src, item)
            d = os.path.join(dst, item)
            if symlinks and os.path.islink(s):
                if os.path.lexists(d):
                    os.remove(d)
                os.symlink(os.readlink(s), d)
                try:
                    st = os.lstat(s)
                    mode = stat.S_IMODE(st.st_mode)
                    os.lchmod(d, mode)
                except:
                    pass # lchmod not available
            elif os.path.isdir(s):
                copytree(s, d, symlinks, ignore)
            else:
                shutil.copy2(s, d)

    def generate_dir_file_map(path):
        dir_files = defaultdict(list)
        with open(path, 'r') as txt:
            files = [l.strip() for l in txt.readlines()]
            for f in files:
                dir_name, id = f.split('/')
                dir_files[dir_name].append(id + '.jpg')
        return dir_files

    train_dir_files = generate_dir_file_map('/content/meta/meta/train.txt')
    test_dir_files = generate_dir_file_map('/content/meta/meta/test.txt')


    def ignore_train(d, filenames):
        print(d)
        subdir = d.split('/')[-1]
        to_ignore = train_dir_files[subdir]
        return to_ignore

    def ignore_test(d, filenames):
        print(d)
        subdir = d.split('/')[-1]
        to_ignore = test_dir_files[subdir]
        return to_ignore

    copytree('/content/images', '/content/food-101/test', ignore=ignore_train)
    copytree('/content/images', '/content/food-101/train', ignore=ignore_test)
    
else:
    print('Train/Test files already copied into separate folders.')

In [5]:
os.path.isdir('/content/food-101/train')
os.path.isdir('/content/food-101/test')

True

In [6]:
from __future__ import print_function
from __future__ import division


from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D, Dropout
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model
from keras.layers import Dense
from keras.layers import Input
import numpy as np
import argparse

In [7]:
def setup_generator(train_path, test_path, batch_size, dimentions):
    train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        train_path,  # this is the target directory
        target_size=dimentions,
        batch_size=batch_size)

    validation_generator = test_datagen.flow_from_directory(
        test_path, # this is the target directory
        target_size=dimentions,
        batch_size=batch_size)

    return train_generator, validation_generator

In [8]:
def load_image(img_path, dimentions, rescale=1. / 255):
    img = image.load_img(img_path, target_size=dimentions)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x *= rescale # rescale the same as when trained

    return x

In [9]:
shape = (224, 224, 3)
X_train, X_test = setup_generator('/content/food-101/train', '/content/food-101/test', 32, shape[:2])

Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


In [10]:
X_train

In [11]:
X_train.num_classes

101

In [12]:
from keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

NUM_CLASSES = 101

CHANNELS = 3

IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'


LOSS_METRICS = ['accuracy']


NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 100
BATCH_SIZE_VALIDATION = 100

# 테스트 배치의 개수
BATCH_SIZE_TESTING = 1

In [13]:
# 모델 제작
model = Sequential()

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
# weights = resnet_weights_path 학습해둔 모델이 있으면 이렇게 불러올 수 있음
model.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE)) 
model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))

# 이미 학습된 영역은 학습하지 않겠다고 설정하는 옵션 
model.layers[0].trainable = False

94773248/94765736 [==============================] - 1s 0us/step


In [14]:
# 모델 구조
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 101)               206949    
Total params: 23,794,661
Trainable params: 206,949
Non-trainable params: 23,587,712
_________________________________________________________________


In [15]:
from tensorflow.keras import optimizers

# optimizer, compile
sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau

def schedule(epoch):
    if epoch < 5:
        return 0.001
    elif epoch < 10:
        return .0002
    elif epoch < 15:
        return 0.00002
    else:
        return .0000005
lr_scheduler = LearningRateScheduler(schedule)
csv_logger = CSVLogger('/content/project.log')
checkpointer = ModelCheckpoint(filepath='/content/project.{epoch:02d}-{val_loss:.2f}.hdf5', verbose=1, save_best_only=True)

model.fit(X_train, validation_data=X_test,
                              epochs=50,
                              steps_per_epoch=X_train.samples//32,
                              validation_steps=X_test.samples//32,
                               callbacks=[lr_scheduler, csv_logger, checkpointer])

Epoch 1/50
2367/2367 [==============================] - 1220s 502ms/step - loss: 4.6587 - accuracy: 0.0122 - val_loss: 4.6225 - val_accuracy: 0.0127

Epoch 00001: val_loss improved from inf to 4.62251, saving model to /content/project.01-4.62.hdf5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/50
2367/2367 [==============================] - 1171s 495ms/step - loss: 4.6301 - accuracy: 0.0139 - val_loss: 4.5997 - val_accuracy: 0.0157

Epoch 00002: val_loss improved from 4.62251 to 4.59967, saving model to /content/project.02-4.60.hdf5
Epoch 3/50
2367/2367 [==============================] - 1139s 481ms/step - loss: 4.6137 - accuracy: 0.0153 - val_loss: 4.5729 - val_accuracy: 0.0192

Epoch 00003: val_loss improved from 4.59967 to 4.57288, saving model to /content/project.03-4.57.hdf5
Epoch 4/50
2367/2367 [==============================] - 1121s 474ms/step - loss: 4.6062 - accuracy: 0.0167 - val_loss: 4.5745 - val_accuracy: 0.0253

Epoch 00004: val_loss did not improve from 4.57288
Epoch 5/50
2367/2367 [==============================] - 1109s 468ms/step - loss: 4.5997 - accuracy: 0.0177 - val_loss: 4.5791 - val_accuracy: 0.0195

Epoch 00005: val_loss did not improve from 4.57288
Epoch 6/50
1509/2367 [==================>...........] - ETA: 5:55 - loss: 4.5474 - accuracy: 0